In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/401'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">401</td>
<td><a href="/box-office-chart/daily/2009/10/16">Oct 16, 2009</a></td>
<td><b><a href="/movie/Where-the-Wild-Things-Are#tab=summary">Where the Wild Things Are</a></b></td>
<td class="data"> $100,000,000</td>
<td class="data"> $77,233,467</td>
<td class="data"> $99,123,656</td>
</tr>
<tr><td class="data">402</td>
<td><a href="/box-office-chart/daily/2006/12/15">Dec 15, 2006</a></td>
<td><b><a href="/movie/Eragon#tab=summary">Eragon</a></b></td>
<td class="data"> $100,000,000</td>
<td class="data"> $75,030,163</td>
<td class="data"> $249,488,115</td>
</tr>
<tr><td class="data">403</td>
<td><a href="/box-office-chart/daily/2014/07/23">Jul 23, 2014</a></td>
<td><b><a href="/movie/Hercules-(2014)#tab=summary">Hercules</a></b></td>
<td class="data"> $100,000,000</td>
<td class="data"> $72,688,614</td>
<td class="

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Oct 16, 2009': ['/box-office-chart/daily/2009/10/16',
  '401',
  'Oct 16, 2009',
  'Where the Wild Things Are',
  '\xa0$100,000,000',
  '\xa0$77,233,467',
  '\xa0$99,123,656'],
 'Dec 15, 2006': ['/box-office-chart/daily/2006/12/15',
  '402',
  'Dec 15, 2006',
  'Eragon',
  '\xa0$100,000,000',
  '\xa0$75,030,163',
  '\xa0$249,488,115'],
 'Jul 23, 2014': ['/box-office-chart/daily/2014/07/23',
  '403',
  'Jul 23, 2014',
  'Hercules',
  '\xa0$100,000,000',
  '\xa0$72,688,614',
  '\xa0$243,388,614'],
 'Dec 10, 2010': ['/box-office-chart/daily/2010/12/10',
  '404',
  'Dec 10, 2010',
  'The Tourist',
  '\xa0$100,000,000',
  '\xa0$67,631,157',
  '\xa0$278,731,369'],
 'Nov 24, 1999': ['/box-office-chart/daily/1999/11/24',
  '405',
  'Nov 24, 1999',
  'End of Days',
  '\xa0$100,000,000',
  '\xa0$66,889,043',
  '\xa0$212,026,975'],
 'Dec 25, 2019': ['/box-office-chart/daily/2019/12/25',
  '406',
  'Dec 25, 2019',
  'Spies in Disguise',
  '\xa0$100,000,000',
  '\xa0$66,757,013',
  '\xa0$165,227,

In [6]:
budget5 = pd.DataFrame(movies).T  #transpose
budget5.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget5.shape

(100, 7)

In [7]:
budget5['budget'] = budget5['budget'].str.replace('$', '')
budget5['budget'] = budget5['budget'].str.replace(',', '')
budget5['budget'] = budget5['budget'].astype(int)

In [8]:
budget5.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Oct 16, 2009",/box-office-chart/daily/2009/10/16,401,"Oct 16, 2009",Where the Wild Things Are,100000000,"$77,233,467","$99,123,656"
"Dec 15, 2006",/box-office-chart/daily/2006/12/15,402,"Dec 15, 2006",Eragon,100000000,"$75,030,163","$249,488,115"
"Jul 23, 2014",/box-office-chart/daily/2014/07/23,403,"Jul 23, 2014",Hercules,100000000,"$72,688,614","$243,388,614"
"Dec 10, 2010",/box-office-chart/daily/2010/12/10,404,"Dec 10, 2010",The Tourist,100000000,"$67,631,157","$278,731,369"
"Nov 24, 1999",/box-office-chart/daily/1999/11/24,405,"Nov 24, 1999",End of Days,100000000,"$66,889,043","$212,026,975"


In [10]:
budget5.to_csv('budget5.csv')